# Objective

recreate what was done on the midterm with the fmi and world sizes, include stuff with the rendevous server, really show any additional setup needed before the actual system

In [1]:
import sys
from time import sleep 

import boto3
import yaml
import json

print(sys.version)

3.11.8 (v3.11.8:db85d51d3e, Feb  6 2024, 18:02:37) [Clang 13.0.0 (clang-1300.0.29.30)]


In [15]:
with open("../keys/aws_credentials.yaml", "r") as f:
    credentials = yaml.safe_load(f)


# Initialize  -- using session authorization
session = boto3.Session(aws_access_key_id=credentials["aws_access_key_id"],
                        aws_secret_access_key=credentials["aws_secret_access_key"],
                        region_name=credentials["region"])

step_function = session.client("stepfunctions", region_name = credentials["region"])

## List Available State Machines 

In [27]:
for res in step_function.list_state_machines()["stateMachines"]:
    print(f'name - {res["name"]} - arn - {res["stateMachineArn"]}')

name - Cosmic_AI_Team_One - arn - arn:aws:states:us-east-1:211125778552:stateMachine:Cosmic_AI_Team_One
name - DS5110StateMachine2 - arn - arn:aws:states:us-east-1:211125778552:stateMachine:DS5110StateMachine2
name - DS5110StateMachine3 - arn - arn:aws:states:us-east-1:211125778552:stateMachine:DS5110StateMachine3
name - DS5110StateMachine4 - arn - arn:aws:states:us-east-1:211125778552:stateMachine:DS5110StateMachine4
name - DataParallel-CosmicAI - arn - arn:aws:states:us-east-1:211125778552:stateMachine:DataParallel-CosmicAI
name - Group2_cosmicai - arn - arn:aws:states:us-east-1:211125778552:stateMachine:Group2_cosmicai
name - Group3_CosmicAI - arn - arn:aws:states:us-east-1:211125778552:stateMachine:Group3_CosmicAI
name - Group_4_CosmicAI - arn - arn:aws:states:us-east-1:211125778552:stateMachine:Group_4_CosmicAI
name - MyStateMachine-Team_1 - arn - arn:aws:states:us-east-1:211125778552:stateMachine:MyStateMachine-Team_1
name - MyStateMachine-e5ydt2afc - arn - arn:aws:states:us-east

## Inspect the State Machine

In [31]:
r = step_function.describe_state_machine(
    stateMachineArn = STATE_MACHINE_ARN
)

r

{'stateMachineArn': 'arn:aws:states:us-east-1:211125778552:stateMachine:DS5110StateMachine4',
 'name': 'DS5110StateMachine4',
 'status': 'ACTIVE',
 'definition': '{"Comment":"A description of my state machine","StartAt":"Lambda init","States":{"Lambda init":{"Type":"Task","Resource":"arn:aws:states:::lambda:invoke","OutputPath":"$.Payload","Parameters":{"FunctionName":"arn:aws:lambda:us-east-1:211125778552:function:fmi_init:$LATEST","Payload":{"world_size":"2","bucket":"group4-s3-bucket","script":"scripts/fmi-execute-test.py"}},"Retry":[{"ErrorEquals":["Lambda.ServiceException","Lambda.AWSLambdaException","Lambda.SdkClientException","Lambda.TooManyRequestsException"],"IntervalSeconds":1,"MaxAttempts":3,"BackoffRate":2}],"Next":"MapState"},"MapState":{"Type":"Map","ResultPath":"$.MapOutput","MaxConcurrency":1000,"Iterator":{"StartAt":"ExtractAndInvokeLambda","States":{"ExtractAndInvokeLambda":{"Type":"Pass","InputPath":"$","ResultPath":"$","Next":"Lambda Invoke FMI"},"Lambda Invoke FMI"

## Show Steps

In [32]:
STATE_MACHINE_ARN = "arn:aws:states:us-east-1:211125778552:stateMachine:DS5110StateMachine4" # DS5110StateMachine4
WORLD_SIZE = 2


# calls the FMI_script
input_payload  = {
  "world_size": f"{WORLD_SIZE}", 
  "bucket": "group4-s3-bucket", # where it dumps
  "script": "scripts/fmi-execute-test.py" # given test script
}

response = step_function.start_execution(
    stateMachineArn = STATE_MACHINE_ARN,
    input = json.dumps(input_payload)
)


In [35]:
response

{'executionArn': 'arn:aws:states:us-east-1:211125778552:execution:DS5110StateMachine4:4c539633-cbdd-4638-94fb-7a51f7e8677c',
 'startDate': datetime.datetime(2024, 11, 28, 14, 13, 24, 308000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '188989cd-3a1f-4ff1-8404-ddd25894026d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '188989cd-3a1f-4ff1-8404-ddd25894026d',
   'date': 'Thu, 28 Nov 2024 19:13:24 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '152',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [39]:
step_function.describe_execution(executionArn = response['executionArn'])

{'executionArn': 'arn:aws:states:us-east-1:211125778552:execution:DS5110StateMachine4:4c539633-cbdd-4638-94fb-7a51f7e8677c',
 'stateMachineArn': 'arn:aws:states:us-east-1:211125778552:stateMachine:DS5110StateMachine4',
 'name': '4c539633-cbdd-4638-94fb-7a51f7e8677c',
 'status': 'SUCCEEDED',
 'startDate': datetime.datetime(2024, 11, 28, 14, 13, 24, 308000, tzinfo=tzlocal()),
 'stopDate': datetime.datetime(2024, 11, 28, 14, 13, 26, 951000, tzinfo=tzlocal()),
 'input': '{"world_size": "2", "bucket": "group4-s3-bucket", "script": "scripts/fmi-execute-test.py"}',
 'inputDetails': {'included': True},
 'output': '{"statusCode":200,"body":[{"S3_BUCKET":"group4-s3-bucket","S3_OBJECT_NAME":"scripts/fmi-execute-test.py","OUTPUT_FILENAME":"/tmp/fmi_test.py","WORLD_SIZE":"2","RANK":"0"},{"S3_BUCKET":"group4-s3-bucket","S3_OBJECT_NAME":"scripts/fmi-execute-test.py","OUTPUT_FILENAME":"/tmp/fmi_test.py","WORLD_SIZE":"2","RANK":"1"}],"MapOutput":["Executed Serverless FMI using Python3.9.19 | packaged b

In [41]:
# show elapsed time
output = step_function.describe_execution(executionArn = response['executionArn'])

print(output['stopDate'] - output['startDate'])

0:00:02.643000


## All Together Now

In [46]:
world_sizes = [num for num in range(2,60, 8)] # jumping every 8 need

world_sizes

[2, 10, 18, 26, 34, 42, 50, 58]

In [65]:
def run_fmi_step_function(world_size :int = WORLD_SIZE) -> int:
    """runs the FMI step function with a variable world size and gives the output"""

    STATE_MACHINE_ARN = "arn:aws:states:us-east-1:211125778552:stateMachine:DS5110StateMachine4" # DS5110StateMachine4

    # calls the FMI_script
    input_payload  = {
    "world_size": f"{world_size}",
    "bucket": "group4-s3-bucket", # where it dumps
    "script": "scripts/fmi-execute-test.py" # given test script
    }

    response = step_function.start_execution(
        stateMachineArn = STATE_MACHINE_ARN,
        input = json.dumps(input_payload)
    )

    print(response)

    sleep(10)
    print(response['executionArn'])
    output = step_function.describe_execution(executionArn = response['executionArn'])

    print(output)
    time_for_run = output['stopDate'] - output['startDate']

    print(f"world size {world_size} - time {time_for_run}")
    return time_for_run



In [66]:
run_fmi_step_function(10)

{'executionArn': 'arn:aws:states:us-east-1:211125778552:execution:DS5110StateMachine4:67ddb9e8-4c38-458a-acc8-a7b221465976', 'startDate': datetime.datetime(2024, 11, 28, 15, 13, 56, 420000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '34714451-8514-4341-beff-9870b4717e32', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '34714451-8514-4341-beff-9870b4717e32', 'date': 'Thu, 28 Nov 2024 20:13:56 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '151', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
arn:aws:states:us-east-1:211125778552:execution:DS5110StateMachine4:67ddb9e8-4c38-458a-acc8-a7b221465976
{'executionArn': 'arn:aws:states:us-east-1:211125778552:execution:DS5110StateMachine4:67ddb9e8-4c38-458a-acc8-a7b221465976', 'stateMachineArn': 'arn:aws:states:us-east-1:211125778552:stateMachine:DS5110StateMachine4', 'name': '67ddb9e8-4c38-458a-acc8-a7b221465976', 'status': 'SUCCEEDED', 'startDate': datetime.datetime(2024, 11, 28, 15, 13, 56, 42000

datetime.timedelta(seconds=3, microseconds=312000)

In [60]:
## TODO figure out how to alter the fmi scripts

arn:aws:states:us-east-1:211125778552:execution:DS5110StateMachine4:ade57b7e-bf9d-454b-b4f3-96577694ffbe
world size 100 - time 0:00:02.996000


datetime.timedelta(seconds=2, microseconds=996000)